# voca quiz mid high 검수
- 문제를 쉽게 풀려고 해라
- 쉽게쉽게쉽게쉽게쉽게 생각...

In [12]:
import pandas as pd
import re

In [7]:
path = "C:\\Users\\WEAVERSMIND\\Desktop\\Weaversmind\\Word\\PreviewQuiz_Word\\MidHigh\\Quiz_Voca_Mid(total).csv"
df = pd.read_csv(path)
df.head()

,Unnamed: 0,fill_in_blank_quiz,korean_translation,answer,hint_eng,hint_kor
0,0,She speaks with a unique ___.,그녀는 독특한 [[강세]]로 말해요.,accent,It is how someone pronounces words differently.,다른 지역 사람들의 말하는 방식이에요.
1,1,She speaks with ___.,그녀는 [[억양]]으로 말해요.,accent,It shows how your voice goes up and down when ...,말할 때 목소리가 위아래로 변해요.
2,2,They ___ on TV.,그들은 TV에 [[광고]]해요.,advertise,Companies tell people about their products.,회사가 사람들에게 제품을 알려주는 것을 말해요.
3,3,The company ___ their product today.,회사에서는 [[광고]]해요.,advertize,You tell people about a product.,사람들에게 제품을 알리는 행동이에요.
4,4,My dad works for ___.,아빠는 [[항공사]]에서 일해요.,airline,It is a company that flies planes.,비행기를 운항하는 회사예요.


In [10]:
sample_df = df[:5]
sample_df.head(1)
data = sample_df

In [73]:
def check_voca_quiz(data:pd.DataFrame) -> pd.DataFrame:
    
    error_state = False

    # 1. korean_translation 확인
    check_symbol_pattern = r"\[\[.*?\]\]"
    blank = r"_+"

    for i in range(len(data)):
        # 1-1 [[]]가 있는지 확인
        if not re.search(check_symbol_pattern, data['korean_translation'][i]):
            error_state = True
            print(f"korean translation {i} 번째 행 {data['answer'][i]} 에 [[]] 오류가 있음")
        
        # 1-2 영어가 있는지 확인
        cleaned_text = re.sub("TV", "",data['korean_translation'][i])
        if re.search(r"[a-zA-Z]", cleaned_text):
            error_state = True
            print(f"korean translation {i} 번째 행 {data['answer'][i]} 에 영어가 있음")

        # 1-3 밑줄이있는지 확인
        underscores = re.findall(blank, data['fill_in_blank_quiz'][i])
        if len(underscores) < 1:
            error_state = True
            print(f"fill in blank quiz {i} 번째 행 {data['answer'][i]} 에 __ 밑줄이 없음")
    
    if error_state==False:
        return f"발견된 에러가 없음!!"

- 다음 부터는 그냥 데이터를 만들자 뭐가 오류인지
    - 이렇게 하니씩 확인하려고 하니 힘듬..

In [74]:
check_voca_quiz(df)

korean translation 1230 번째 행 height 에 영어가 있음
korean translation 1545 번째 행 share 에 영어가 있음
korean translation 2371 번째 행 worth 에 [[]] 오류가 있음
korean translation 3415 번째 행 campfire 에 [[]] 오류가 있음
korean translation 4297 번째 행 both A and B 에 영어가 있음
korean translation 5215 번째 행 mix A with B 에 영어가 있음
korean translation 5510 번째 행 part 에 [[]] 오류가 있음
korean translation 5839 번째 행 read 에 영어가 있음
korean translation 6176 번째 행 skirt 에 [[]] 오류가 있음


In [25]:
check_symbol_pattern = r"\[\[.*?\]\]"
blank = r"_+"

for i in range(len(data)):
    if not re.search(check_symbol_pattern, data['korean_translation'][i]):
        print(f"korean translation {i} 번째 행에 [[]] 오류가 있음")

    if re.search(r"[a-zA-Z]", data['korean_translation'][i]):
        print(f"korean translation {i} 번째 행에 영어가 있음")

    underscores = re.findall(blank, data['fill_in_blank_quiz'][i])
    if len(underscores) < 1:
        print(f"fill in blank quiz {i} 번째 행에 __ 밑줄이 없음")
    
    

korean translation 2 번째 행에 영어가 있음


In [23]:
data['korean_translation']

0    그녀는 독특한 [[강세]]로 말해요.
1       그녀는 [[억양]]으로 말해요.
2       그들은 TV에 [[광고]]해요.
3         회사에서는 [[광고]]해요.
4      아빠는 [[항공사]]에서 일해요.
Name: korean_translation, dtype: object

In [24]:
data['fill_in_blank_quiz']

0           She speaks with a unique ___.
1                    She speaks with ___.
2                         They ___ on TV.
3    The company ___ their product today.
4                   My dad works for ___.
Name: fill_in_blank_quiz, dtype: object